In [1]:
import requests
from pymongo import MongoClient
from urllib.parse import urljoin
from pprint import pprint
import nltk
import json

# Retrieve Lyrics Via API

In [2]:
# DB CONFIG
client = MongoClient('mongo', 27017)
db = client.music_db
COLLECTION = 'pop-songs'
songs_collection = db[COLLECTION]

In [3]:
# Pull all songs from DB
songs = list(songs_collection.find())

In [4]:
MUSICMATCH_API = '309401c7a9740d7c3bfbc94bca9a911c'
APISEED_API = 'ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0'

In [5]:
class ApiSeed:
    BASE_URL = 'https://orion.apiseeds.com/api/music/lyric/'
    
    def __init__(self, api_key):
        """ Load automatically translates responses from HTTP request objects into Dictionaries """
        self.api_key = api_key
        self.load = True
    
    def _api_url(self, url):
        url = urljoin(self.BASE_URL, url)
        url = urljoin(url, '?apikey=ICmmIMXiTjyqhQjXC0HX4oqZP3vEOp9zr2prrrM5MUDe4hWri6gVTCebOCkARWi0')
        return url
    
    def _retrieve(self, url):
        response = requests.get(self._api_url(url))
        if self.load:
            response = json.loads(response.content)
        return response
    
    def get_track(self, artist, track):
        return self._retrieve(f'{artist}/{track}')
    

apiseed = ApiSeed(APISEED_API)
res = apiseed.get_track('Kanye West', 'Everything I am')

# Insert Lyrics into DB

In [ ]:
import time
def insert_into_db():
    for song in songs:
        pass
    
def update_db(document_id, lyrics: str):
    document = songs_collection.find_one_and_update({"_id": document_id}, 
                                         {"$set": {"lyrics": lyrics}})
    return document

def retrieve_lyrics(track: dict):
    """ Takes mongodb document and gets lyrics"""
    res = apiseed.get_track(track['artist'], track['title'])
    return res

def find_and_update_lyrics(track: dict):
    """ Updates track with lyrics """
    lyrics = retrieve_lyrics(track)
    track['lyrics'] = lyrics
    return track

        
start_index = 837
all_tracks = []
for i, track in enumerate(songs[start_index:], start_index):
    try:
        track = find_and_update_lyrics(track)

        if track.get('lyrics').get('error'):
            print("Error", end=" "); 
        else:
            print("Success", end=" ")
            
    except (json.JSONDecodeError, Exception) as e:
        print(e)
        track['lyrics'] = 'null'
        
    update_db(track['_id'], track['lyrics'])
    print(i, track['title'], end = " | ")
    time.sleep(.33)



Success 837 With Ur Love | Error 838 Don't You Worry Child | Success 839 I Knew You Were Trouble. | Error 840 If I Lose Myself | Success 841 It's Time | Error 842 Girl On Fire | Success 843 Kiss You | Success 844 Locked Out Of Heaven | Error 845 Ho Hey | Success 846 Try | Success 847 Forever Now | Success 848 Harlem Shake | Success 849 Catch My Breath | Error 850 Beauty And A Beat | Success 851 Gold | Success 852 Hey Porsche | Success 853 Wanted | Success 854 Simply Amazing | Success 855 Don't Stop The Party | Error 856 Who Booty | Error 857 Va Va Voom | Error 858 Hall Of Fame | Success 859 Remember When (Push Rewind) | Error 860 C'mon Let Me Ride | Success 861 The A Team | Success 862 I Cry | Success 863 Diamonds | Error 864 Die Young | Success 865 Little Things | Success 866 Give Me Your Hand (Best Song Ever) | Error 867 Turn Around | Error 868 Rest Of My Life | Success 869 One More Night | Success 870 Lessons In Love (All Day, All Night) | Success 871 Home | Success 872 Feel Again |

In [ ]:
songs_collection = db[COLLECTION]

def has_lyrics(song):
    if not song.get('lyrics'):
        return False
    
    if song.get('lyrics') == 'null':
        return False

    if song.get('lyrics').get('error'):
        return False
        
    return True


In [ ]:
songs = [song for song in songs_collection.find() if has_lyrics(song)]

In [ ]:
len(songs)